In [16]:
import librosa  # for audio processing
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile  # for audio processing
from numpy.lib.stride_tricks import as_strided
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Audio
import sklearn
import pandas as pd
import json
import os
from os.path import exists
import warnings
warnings.filterwarnings("ignore")
# from cleaner import CleanDataFrame

In [17]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [18]:
import os
os.chdir("/content/drive/My Drive/SWAHILI")
os.listdir()

['LM',
 'README',
 'data',
 'kaldi-scripts',
 'lang',
 'swahili-dataset-CCO-license.png',
 'train_corpus.json',
 'valid_corpus.json',
 'model_1.png',
 'models']

### Preprocessing steps:
1. Load labels
2. sample each audio at 44100Hz
3. Convert mono to stereo
4. Resize audios
5. Generate a pandas dataframe
6. save preprocessed audios and transcritions to a new folder
6. Split the data to train and valid corpus
7.  save train and valid corpus

Defining file Paths

In [22]:
train_dataset_location = 'train/SWA/'
train_wav_location = os.path.join('/content/drive/My Drive/SWAHILI/data/train/wav/','SWA/')
train_changed_wav_location = os.path.join('/content/drive/My Drive/SWAHILI/data/train/wav/','changed_wav/')
train_txt_location =("/content/drive/My Drive/SWAHILI/data/train/text.txt")
test_dataset_location = 'test/' 
lexicon_location = 'train/SWA/'

In [20]:
def convert_to_sterio(audio: np.array) -> np.array:
        if len(audio.shape) == 1:
            sterio = np.stack([audio, audio], axis=1)
            return sterio
        return audio


from scipy.io import wavfile

def resize_audio(audio: np.array, size: int) -> np.array:
        """
        This resizes all input audio to a fixed sample size.
        It helps us to have a consistent data shape

        Args:
            audio: This is the audio sample as a numpy array
        """
        resized = librosa.util.fix_length(audio, size, axis=0)
        print(f"Audio resized to {size} samples")
        return resized
# changed = convert_to_sterio(samples)
# changed.shape  
# resized = resize_audio(changed,200000)
# print(resized.shape)
# print(resized.T.shape)
# wavfile.write(os.path.join(train_changed_wav_location, 'trial.wav'), sample_rate, resized)
# ipd.Audio(resized.T, rate=sample_rate)
# # mfcc = librosa.feature.mfcc(y=changed, sr=sample_rate)
# # mfcc.shape
# samples, sample_rate = librosa.load(train_changed_wav_location+'trial.wav' , sr=44100)
# samples.shape
# wav_roll = np.roll(samples, int(sample_rate/10))

def meta_data(trans, path):
        target = []
        features = []
        mode = []
        rmse = []
        spec_cent = []
        spec_bw = []
        rolloff = []
        zcr = []
        mfcc = []
        rate = []
        filenames = []
        duration_of_recordings = []
        for index, k in enumerate(trans):
            if True:
                filename = path + k + ".wav"
                next_file_name = path + k + "changed.wav"
                if exists(filename):
                    audio, fs = librosa.load(filename, sr=44100)
                   
                    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=fs)
                    rmse.append(np.mean(librosa.feature.rms(y=audio)))
                    spec_cent.append(
                        np.mean(librosa.feature.spectral_centroid(y=audio, sr=fs)))
                    spec_bw.append(
                        np.mean(librosa.feature.spectral_bandwidth(y=audio, sr=fs)))
                    rolloff.append(
                        np.mean(librosa.feature.spectral_rolloff(y=audio, sr=fs)))
                    zcr.append(
                        np.mean(librosa.feature.zero_crossing_rate(audio)))
                    mfcc.append(np.mean(librosa.feature.mfcc(y=audio, sr=fs)))
                    duration_of_recordings.append(float(len(audio)/fs))
                    rate.append(fs)
                    changed = convert_to_sterio(audio)
                    audio = resize_audio(changed,200000)
                   
                    # stereo = change_channel_to_stereo(filename, next_file_name)
                    # resized = self.resize_audio(audio,200000)
                    split_array = str(filename).split('/')
                    filename = '../data/train/changed_wav/' + str(split_array[len(split_array)-1])
                    wavfile.write(os.path.join(train_changed_wav_location, str(split_array[len(split_array)-1]) ), fs, audio)
                    filenames.append(filename)
                    mode.append('mono')  # if stereo == 1 else 'stereo')
                    lable = trans[k]
                    target.append(lable)
        # self.logger.info(f"Meta Data Generated For {len(filenames)} Audios")
        return filenames, target, duration_of_recordings, mode, rmse, spec_cent, spec_bw, rolloff, zcr, mfcc, rate

In [23]:
def loaderTrans(filename: str):
        """
        # Loads the audio file and returns the audio data and sample rate
        # param filename: The path to the txt file
        # @return: The audio data and sample rate
        #
        """
        name_to_text = {}
        with open(filename, encoding="utf-8") as f:
            for line in f:
                f.readline()
                name=line.split("\t")[0]
                text=line.split("\t")[1]
                text=text.replace('\n','')
                name_to_text[name]=text
                # self.logger.info(f"Training data loaded: {name}")
        return name_to_text
transcription = loaderTrans(train_txt_location)

In [24]:
filenames, target, duration_of_recordings,mode ,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc,rate = meta_data(transcription, train_wav_location)
data = pd.DataFrame({'key': filenames, 'text': target,
                    'duration': duration_of_recordings, 'mode': mode , 'rate': rate ,'rmse': rmse,'spec_cent' :spec_cent,'spec_bw': spec_bw,"rolloff" :rolloff,"zcr": zcr,"mfcc": mfcc})
data.head()

Streaming output truncated to the last 5000 lines.
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 200000 samples
Audio resized to 2000

,key,text,duration,mode,rate,rmse,spec_cent,spec_bw,rolloff,zcr,mfcc
0,../data/train/changed_wav/SWH-05-20101106_16k-...,rais wa tanzania jakaya mrisho kikwete,3.14,mono,44100,0.168974,1845.298698,1677.769444,3474.950451,0.048648,-1.024828
1,../data/train/changed_wav/SWH-05-20101106_16k-...,inayokutangazia moja kwa moja kutoka jijini da...,3.65,mono,44100,0.135938,1893.996162,1744.379220,3657.089844,0.044638,-6.544104
2,../data/train/changed_wav/SWH-05-20101106_16k-...,wakipiga kura ya maoni ilikufanya mabadiliko ya,2.94,mono,44100,0.143796,1667.030359,1689.968234,3377.152282,0.038920,-7.525208
3,../data/train/changed_wav/SWH-05-20101106_16k-...,walipata fursa ya kutumia haki yao ya msingi,2.62,mono,44100,0.130586,2250.586209,1745.139667,4001.936273,0.076340,-7.392262
4,../data/train/changed_wav/SWH-05-20101106_16k-...,na rais aliyetangulia henry konan berdi,3.53,mono,44100,0.162911,1900.782509,1652.913709,3475.388384,0.049550,-6.657930


In [25]:
# Checking size to determine propotion of splitiing
data.shape

(5090, 11)

In [26]:
train_json = data[3000:].to_dict(orient='records')
with open("train_corpus.json", "w", encoding='UTF-8') as export_file:
            json.dump(train_json, export_file, indent=4,
                     sort_keys=True, ensure_ascii=False)
valid_json = data[:3001].to_dict(orient='records')
with open("valid_corpus.json", "w", encoding='UTF-8') as export_file:
            json.dump(valid_json, export_file, indent=4,
                     sort_keys=True, ensure_ascii=False)